# Decision Trees

In [1]:
import numpy as np

In [ ]:
def entropy(y):
    # find no. of occurences for each unique y-value
    hist = np.bincount(y)
    # calculate the probability of each class
    ps = hist / len(y)
    # return negative sum of products of each probability that is greater than 0 and the probability's log
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

class Node:
    # initiate class and attributes
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None